In [2]:
import pandas as pd
import numpy as np

In [3]:
# Read CSV into pandas dataframe.
df = pd.read_csv('trans_retail_prices.csv')

In [4]:
# Preview df
df.head()

,Calendar years,Unnamed: 1,European Union,Austria,Belgium,Bulgaria,Cyprus,Czech Republic,Denmark,Finland,...,Slovenia,Spain,Sweden,United Kingdom 1,Unnamed: 26,Japan,Norway,Russian Federation,Switzerland,USA
0,1990,NaN,NaN,4.90,3.27,NaN,2.83,NaN,3.81,2.98,...,NaN,3.62,3.43,10.55,NaN,10.26,3.31,NaN,4.83,2.97
1,1991,NaN,NaN,4.57,2.92,NaN,2.80,NaN,3.67,2.72,...,NaN,3.50,3.29,10.41,NaN,11.88,3.16,NaN,4.36,2.81
2,1992,NaN,NaN,4.99,3.05,NaN,2.87,NaN,3.79,2.46,...,NaN,3.50,3.11,10.09,NaN,12.62,3.12,NaN,4.52,2.58
3,1993,NaN,NaN,4.97,2.78,NaN,2.60,NaN,3.48,1.94,...,NaN,2.79,2.65,8.44,NaN,14.57,2.46,NaN,4.27,2.47
4,1994,NaN,NaN,4.58,3.42,NaN,3.18,NaN,4.45,2.73,...,NaN,2.76,3.79,11.36,NaN,14.69,3.29,NaN,4.50,3.40


## Peek Analysis
It is immediately evident the transpose in excel created some minor problems: 

(a) the data set contains columns intended for naming elements in sets (i.e. 'European Union').

(b) there are unnamed rows with nothing in them.

(c) there are columns with names appended 1 (i.e. Malta 1) which represent special types of coffee (soluble).

In [5]:
# Dtypes look clean
df.dtypes

Calendar years           int64
Unnamed: 1             float64
European Union         float64
   Austria             float64
   Belgium             float64
   Bulgaria            float64
   Cyprus              float64
   Czech Republic      float64
   Denmark             float64
   Finland             float64
   France              float64
   Germany             float64
   Hungary             float64
   Italy               float64
   Latvia              float64
   Lithuania           float64
   Luxembourg          float64
   Malta 1             float64
   Netherlands         float64
   Poland              float64
   Portugal            float64
   Slovakia            float64
   Slovenia            float64
   Spain               float64
   Sweden              float64
   United Kingdom 1    float64
Unnamed: 26            float64
Japan                  float64
Norway                 float64
Russian Federation     float64
Switzerland            float64
USA                    float64
dtype: o

In [6]:
df.columns = list(df.columns)
df.columns = df.columns.str.replace("   ","")
df.columns = df.columns.str.replace(" 1",".1")

## Cleaning Data
### Investigating columns
#### Observations
1. Unnamed and other problem columns have dtype int64 but actually contain NaNs.
    
    a. We will want to drop Unnamed columns but keep columns referencing coffee type.

In [7]:
# Get column names and turn into a list for iteration
cols = list(df.columns)

count = 0
lst_garb = []

# Count and print list of unwanted column names
for col in cols:
    if "Unnamed" in col:
        count += 1
        lst_garb.append(col)
        
print(count)
print(lst_garb)

2
['Unnamed:.1', 'Unnamed: 26']


In [8]:
# df1 dropped df cols
df1 = df.drop(columns=lst_garb)
df1.head()
len(df1.columns)

30

In [9]:
# assign cols with new df1 columns
cols = df1.columns

# 29 rows of data expected
exp_row = len(df1.index)

# Empty lists for iteration and appending
lst_col_drop = []
lst_col_miss = ['Calendar years']
lst_col_good = []

# Set of lists to create a df for missing
lst_key_miss = []
lst_val_miss = []

# Collect columns with missing data for a dataframe
lst_dict_miss = []

for idx, col in enumerate(cols):
    count_na = 0
    for row in df1[col]:
        if row == '' or pd.isnull(row):
            count_na += 1
            
    # Country has any missing values append
    if count_na > 0 and count_na < exp_row: 
        key = col
        val = count_na
        
        lst_key_miss.append(key)
        lst_val_miss.append(val)
        
        lst_col_miss.append(col)
        
        print(f"""
        x{'-'*4}(Missing Values){'-'*4}x
        {key} : {val}
        """)
        
    # Column only has missing values append
    if count_na == exp_row:
        key = col
        val = count_na
        
        lst_col_drop.append(col)
        
        print(f"""
        #{'-'*4}(Region Category){'-'*4}#
        {key} : {val}
        """)
    
    # Country has no missing values append
    if count_na == 0:
        key = col
        val = count_na
        
        lst_col_good.append(col)
        
        print(f"""
        ${'-'*4}(Good Series){'-'*4}$
        {key} : {val}
        """)

    
print(f"""
=====
Summary
-----
{len(cols)} total columns.

{len(lst_col_drop)} columns to drop.
{len(lst_key_miss)} columns with missing data.
{len(lst_col_good)} columns with complete data.
""")


        $----(Good Series)----$
        Calendar years : 0
        

        #----(Region Category)----#
        European Union : 29
        

        $----(Good Series)----$
        Austria : 0
        

        x----(Missing Values)----x
        Belgium : 5
        

        x----(Missing Values)----x
        Bulgaria : 12
        

        $----(Good Series)----$
        Cyprus : 0
        

        x----(Missing Values)----x
        Czech Republic : 7
        

        $----(Good Series)----$
        Denmark : 0
        

        $----(Good Series)----$
        Finland : 0
        

        $----(Good Series)----$
        France : 0
        

        $----(Good Series)----$
        Germany : 0
        

        x----(Missing Values)----x
        Hungary : 4
        

        $----(Good Series)----$
        Italy : 0
        

        x----(Missing Values)----x
        Latvia : 3
        

        x----(Missing Values)----x
        Lithuania : 7
        

        x----(Missing Valu

=====
Summary
-----
30 total columns.

1 columns to drop.
14 columns with missing data.
15 columns with complete data.

In [10]:
# Assign regions with missing values to df
df_missing_list = pd.DataFrame(data=({
    'Country': lst_key_miss, 
    'Missing_Rows': lst_val_miss
}))

In [11]:
# df record of missing rows with counts by region
df_missing_list = df_missing_list.sort_values('Missing_Rows', ascending=False).reset_index(drop=True)

# df of complete data 
df_good = df1[lst_col_good]

# df of columns with missing data
df_missing = df1[lst_col_miss]

# df of columns dropped that contained header info
df_drop = df1[lst_col_drop]

In [12]:
# Soluble extraction for complete data
# Get column names and turn into a list for iteration

def extract_sol(df,string):
    f"""
    Takes in dataframe, checks column names against substring,
    and filters and drops.
    """
    
    cols = list(df.columns)

    count = 0
    lst = []

    # Count and print list of unwanted column names
    for col in cols:
        if string in col:
            count += 1
            lst.append(col)

    print(count)
    print(lst)

    # Drop soluble
    df.drop(columns=(lst))
    return df

def store_sol(df,string):
    f"""
    Takes in dataframe, checks column names against substring,
    and filters and drops.
    """
    
    cols = list(df.columns)

    count = 0
    lst = []

    # Count and print list of unwanted column names
    for col in cols:
        if string in col:
            count += 1
            lst.append(col)

    print(count)
    print(lst)

    return df[lst]

In [13]:
df_good = extract_sol(df_good,".1")

1
['United Kingdom.1']


In [14]:
df_missing = extract_sol(df_missing,".1")

1
['Malta.1']


In [15]:
df_join = df_good.join(df_missing, lsuffix='_dup')
df_join_col = pd.DataFrame(list(df_join.columns), columns=['Country'])
df_join_col.to_csv('retail_col_names.csv', index=False)

In [16]:
# Save to csv
df_missing_list.to_csv('retail_missing_data_list.csv')

In [17]:
list = df_good.columns

res = [sub.replace(' ', '_') for sub in list] 
df_good.columns = res 
df_good.columns

Index(['Calendar_years', 'Austria', 'Cyprus', 'Denmark', 'Finland', 'France',
       'Germany', 'Italy', 'Netherlands', 'Portugal', 'Spain', 'Sweden',
       'United_Kingdom.1', 'Japan', 'Norway'],
      dtype='object')

In [16]:
df_good.to_csv('retail_complete_data.csv', index=False)

In [17]:
df_missing.to_csv('retail_missing_data.csv')

In [18]:
df_drop.to_csv('ret_orginal_category_data.csv')